In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.5/193.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.8/846.8 kB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 50.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 98.2 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=Au

In [2]:
from collections import OrderedDict
import numpy as np

import torch

import matplotlib.pyplot as plt
from google.colab import drive

import gym
from stable_baselines3.common.vec_env import DummyVecEnv

import imageio
from base64 import b64encode
from IPython.display import HTML

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [3]:
from train import AgentTrainer
from utils import image_process
from env import Stack

class SerialModelTrain():
  def __init__(self):
    self.num_episode = 10
    self.initial_memory_size = 5
    self.episode_rewards = []
    self.num_average_epidodes = 100
    self.save_every = 100
    self.batch_size=3072
    self.max_steps = 100
    self.envs = Stack('Panda')
    self.robot_state_shape = 7
    self.action_shape = self.envs[0].robots[0].robot_model.dof + self.envs[0].robots[0].gripper.dof
    self.agent_trainer = AgentTrainer(self.action_shape, self.robot_state_shape)

    self.evaluate_interval = 10
    self.reward_window = 20
    self.reward_values = [0]*self.reward_window 
    self.averge_reward = 0 

  def init_buffer(self):
    """ Initially, put the data into the replay buffer when an action with noise was taken """
    state = OrderedDict()
    next_state = OrderedDict()
    done = False
    reward = 0 

    state = self.env.reset()

    for step in range(self.initial_memory_size):
        if step % self.max_steps == 0:
          state = self.env.reset()

        action = np.random.randn(self.action_shape) # sample random action
        next_state, reward, done, info = self.env.step(action)

        # verify the following in the server
        vision = image_process(state['frontview_image'])
        vision_next = image_process(next_state['frontview_image'])
        action = torch.tensor(action, dtype=torch.float32)
        action = action.squeeze(0)

        self.agent_trainer.pri_buffer.store(state, action, reward, next_state, done)
        state = next_state

    print('%d Data collected' % self.initial_memory_size)


  def train_model(self):
    state = OrderedDict() 
    next_state = OrderedDict() 
    done = False 
    reward = 0 

    for episode in range(self.num_episode):
      state = self.env.reset()
      episode_reward = 0
      for t in range(self.max_steps):
        vision = image_process(state['frontview_image'])

        robot_state = [
            np.array(state['robot0_eef_pos'], dtype=np.float32).flatten(),
            np.array(state['robot0_eef_quat'], dtype=np.float32).flatten()
        ]

        robot_state = np.concatenate(robot_state)
        robot_state = torch.tensor(robot_state, dtype=torch.float32)

        action = self.agent_trainer.get_action(vision, robot_state)
        next_state, reward, done, info = self.env.step(action[0])

        if len(self.reward_value)>=20:
          self.reward_value.pop(0)
          self.reward_value.append(reward)

        # verify the following in the server
        vision_next = image_process(next_state['frontview_image'])
        vision = vision.squeeze(0)
        vision_next = vision_next.squeeze(0)
        action = torch.tensor(action, dtype=torch.float32)
        action = action.squeeze(0)

        self.agent_trainer.pri_buffer.store(state, action, reward, next_state, done)
        state = next_state
        if any(done):
          break

      episode_reward += reward

      if episode % 5 == 0:
        self.agent_trainer.update()
      self.episode_rewards.append(episode_reward)
      if episode % self.evaluate_interval == 0:
        self.averge_reward = np.mean(self.reward_value)
        print(self.averge_reward)
      if episode % 100 == 0:
        print("Episode %d finished | Episode reward %f" % (episode, episode_reward))
      if episode % self.save_every == 0:
        self.agent_trainer.save_checkpoint(episode)

    self.env.close()

  def plot(self):
    # Compute the moving average of cumulative rewards
    moving_average = np.convolve(self.episode_rewards, np.ones(self.num_average_epidodes)/self.num_average_epidodes, mode='valid')
    plt.plot(np.arange(len(moving_average)),moving_average)
    plt.title('Average rewards in %d episodes' % self.num_average_epidodes)
    plt.xlabel('episode')
    plt.ylabel('rewards')
    plt.show()

  def test_model(self):
    env = Stack('Panda')
    states = env.reset()
    self.agent_trainer.load_checkpoint('/content/drive/My Drive/check_point')

    frames = []
    for i in range(100):
      frames.append(states['frontview_image'])  # Append the image to the frames list
      obs = image_process(states['frontview_image'])
      action = self.agent_trainer.get_action(obs)

      next_state, reward, done, info = env.step(action[0])
      state = next_state


    imageio.mimwrite('robosuite_video.mp4', frames, fps=20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/robosuite/__init__.py:7: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  ROBOSUITE_DEFAULT_LOGGER.warn("No private macro file found!")
[robosuite WARNING] No private macro file found! (__init__.py:7)
[robosuite WARNING] It is recommended to use a private macro file (__init__.py:8)
[robosuite WARNING] To setup, run: python /usr/local/lib/python3.10/dist-packages/robosuite/scripts/setup_macros.py (__init__.py:9)


## Train agent

In [ ]:
# drive.mount('/content/drive')

serial_train = SerialModelTrain()
serial_train.init_buffer()
serial_train.train_model()
serial_train.plot()


cuda


## Test Model

In [ ]:
serial_train.test_model()

In [ ]:
""" Load video and encode it in base64 format """
video_path = 'robosuite_video.mp4'
video_data = open(video_path, 'rb').read()
video_encoded = b64encode(video_data).decode()

# Display video using HTML
HTML(f"""
<video width="640" height="480" controls>
  <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
</video>
""")